In [59]:
import pandas as pd
import numpy as np

In [60]:
ch1 = pd.read_csv("CH1.csv")
ch2 = pd.read_csv("CH2.csv")
ch3 = pd.read_csv("CH3.csv")
ch4 = pd.read_csv("CH4.csv")

ch1_X = ch1.select_dtypes(exclude = ["int64", "object"]).copy()
ch2_X = ch2.select_dtypes(exclude = ["int64", "object"]).copy()
ch3_X = ch3.select_dtypes(exclude = ["int64", "object"]).copy()
ch4_X = ch4.select_dtypes(exclude = ["int64", "object"]).copy()

ch1_X.drop(["0"], axis = 1, inplace = True)
ch2_X.drop(["0"], axis = 1, inplace = True)
ch3_X.drop(["0"], axis = 1, inplace = True)
ch4_X.drop(["0"], axis = 1, inplace = True)

Y = ch1["label"]

In [61]:
def getIndicesToBeRemoved(data):
    fft = data.select_dtypes("float64").T

    indices = np.where(np.max(fft) == np.inf)[0]
    indices = np.concatenate((indices, np.where(data.isna().any(axis = 1))[0]))

    return indices


In [62]:
data = pd.concat([ch1_X, ch2_X, ch3_X, ch4_X], axis = 1)
i = getIndicesToBeRemoved(data)

In [63]:
ch1_X = ch1_X.drop(list(set(i))).reset_index(drop=True)

currentData = pd.concat([ch2_X, ch3_X, ch4_X], axis = 1) 
currentData = currentData.drop(list(set(i))).reset_index(drop=True)

Y = Y.drop(list(set(i))).reset_index(drop=True)

### Normalization

In [64]:
def normalize(data):
    norm = np.reshape(np.linalg.norm(data, axis = 1), (-1,1))

    return data / norm

#### Flux

In [65]:
normalized_flux = normalize(ch1_X)

#### Current

In [66]:
normalized_current = normalize(currentData)

### Splitting

#### Flux

In [67]:
from sklearn.model_selection import train_test_split

flux_x_train, flux_x_test, flux_y_train, flux_y_test = train_test_split(normalized_flux, Y, test_size=0.2, random_state=0, stratify=Y)

#### Current

In [68]:
from sklearn.model_selection import train_test_split

current_x_train, current_x_test, current_y_train, current_y_test = train_test_split(normalized_current, Y, test_size=0.2, random_state=0, stratify=Y)

### Models

#### Flux Model

In [69]:
import tensorflow as tf

from keras.layers import Dense, Dropout
from keras import Sequential
from keras.losses import SparseCategoricalCrossentropy

flux_model = Sequential([Dense(2500, activation = "relu"),
                    Dropout(0.2),
                    Dense(1200, activation = "relu"),
                    Dropout(0.2),
                    Dense(600, activation = "relu"),
                    Dropout(0.2),
                    Dense(300, activation = "relu"),
                    Dropout(0.2),
                    Dense(150, activation = "relu"),
                    Dropout(0.2),
                    Dense(75, activation = "relu"),
                    Dropout(0.2),
                    Dense(30, activation = "relu"),
                    Dropout(0.2),
                    Dense(7, activation = "softmax")])

flux_model.compile(loss=SparseCategoricalCrossentropy(), optimizer = "adam",metrics=["accuracy"])

In [70]:
flux_model.fit(flux_x_train, flux_y_train, 
                epochs = 50, 
                validation_data = [flux_x_test, flux_y_test])

Epoch 1/50
65/65 [==============================] - 1s 10ms/step - loss: 1.7819 - accuracy: 0.2162 - val_loss: 1.5599 - val_accuracy: 0.2838
Epoch 2/50
65/65 [==============================] - 1s 8ms/step - loss: 1.4966 - accuracy: 0.3296 - val_loss: 1.4108 - val_accuracy: 0.3764
Epoch 3/50
65/65 [==============================] - 1s 8ms/step - loss: 1.3433 - accuracy: 0.4020 - val_loss: 1.2670 - val_accuracy: 0.4015
Epoch 4/50
65/65 [==============================] - 1s 8ms/step - loss: 1.2917 - accuracy: 0.4025 - val_loss: 1.1998 - val_accuracy: 0.4208
Epoch 5/50
65/65 [==============================] - 0s 7ms/step - loss: 1.2528 - accuracy: 0.4334 - val_loss: 1.2052 - val_accuracy: 0.4170
Epoch 6/50
65/65 [==============================] - 0s 8ms/step - loss: 1.2397 - accuracy: 0.4310 - val_loss: 1.1622 - val_accuracy: 0.4382
Epoch 7/50
65/65 [==============================] - 0s 8ms/step - loss: 1.1860 - accuracy: 0.4595 - val_loss: 1.0557 - val_accuracy: 0.5560
Epoch 8/50
65/65 [=

#### Current Model

In [71]:
current_model = Sequential([Dense(2500, activation = "relu"),
                    Dense(1200, activation = "relu"),
                    Dense(600, activation = "relu"),
                    Dense(300, activation = "relu"),
                    Dense(150, activation = "relu"),
                    Dense(75, activation = "relu"),
                    Dense(30, activation = "relu"),
                    Dense(7, activation = "softmax")])

current_model.compile(loss=SparseCategoricalCrossentropy(), optimizer = "rmsprop",metrics=["accuracy"])

In [72]:
current_model.fit(current_x_train, current_y_train,
                epochs = 200,
                validation_data = [current_x_test, current_y_test])

Epoch 1/200


ResourceExhaustedError: Graph execution error:

Detected at node 'RMSprop/RMSprop/update_4/mul_2' defined at (most recent call last):
    File "c:\Users\Lohith\miniconda3\envs\tensor\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\Lohith\miniconda3\envs\tensor\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\Lohith\miniconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Lohith\miniconda3\envs\tensor\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\Lohith\miniconda3\envs\tensor\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Lohith\miniconda3\envs\tensor\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Lohith\miniconda3\envs